In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_vertical_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,26860.46,-0.09,0.03,-0.05,-0.03,-0.0,-0.0,0.00
3,-0.09,6461.75,-0.00,0.03,0.00,0.0,0.0,-0.00
4,0.03,-0.00,1255.47,-0.02,-0.00,0.0,-0.0,0.00
5,-0.05,0.03,-0.02,237.58,0.00,0.0,0.0,-0.00
6,-0.03,0.00,-0.00,0.00,40.33,-0.0,0.0,0.00
7,-0.00,0.00,0.00,0.00,-0.00,6.1,-0.0,-0.00
8,-0.00,0.00,-0.00,0.00,0.00,-0.0,0.9,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.0,0.0,0.12


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,0.00000,-0.00002,-0.00003,-0.00001,-0.00002,0.00000
3,-0.00001,1.00000,-0.00000,0.00002,0.00001,0.00002,0.00000,-0.00001
4,0.00000,-0.00000,1.00000,-0.00004,-0.00002,0.00004,-0.00001,0.00001
5,-0.00002,0.00002,-0.00004,1.00000,0.00002,0.00000,0.00001,-0.00002
6,-0.00003,0.00001,-0.00002,0.00002,1.00000,-0.00001,0.00002,0.00003
7,-0.00001,0.00002,0.00004,0.00000,-0.00001,1.00000,-0.00001,-0.00001
8,-0.00002,0.00000,-0.00001,0.00001,0.00002,-0.00001,1.00000,0.00001
9,0.00000,-0.00001,0.00001,-0.00002,0.00003,-0.00001,0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.16802885943402765

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[9.03027100e+07 5.11219842e+06 1.87719913e+05 6.45287878e+03
 1.74365042e+02 3.55009558e+00 5.91536911e-02 4.08453682e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999315,-0.036396,-0.006619,-0.001252,-0.000213,-0.000032,-0.000005,-6.366088e-07
1,0.036143,0.998754,-0.033858,-0.006129,-0.001039,-0.000157,-0.000023,-3.209463e-06
2,0.007777,0.033323,0.998616,-0.039416,-0.006412,-0.000964,-0.000144,-1.936562e-05
3,0.001765,0.007333,0.038855,0.998233,-0.043852,-0.006396,-0.000948,-1.290801e-04
4,0.000375,0.001552,0.008003,0.043165,0.997668,-0.051695,-0.007436,-1.006782e-03
5,0.000075,0.000311,0.001599,0.008467,0.050642,0.995760,-0.075663,-9.933582e-03
6,0.000017,0.000068,0.000351,0.001855,0.010934,0.073149,0.988006,-1.355334e-01
7,0.000005,0.000019,0.000097,0.000512,0.003012,0.019938,0.134396,9.907225e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006852666357920478,
 0.0012464547839127205,
 0.0013842751867444436,
 0.0017672553916144862,
 0.0023319536603396696,
 0.004240285960546153,
 0.011993559294262712,
 0.009277542578419729]